## MariaDB(MySQL)와 파이썬 연동하기
- pymysql : 파이썬-DB 연동을 지원하는 라이브러리
- 파이썬으로 DB를 변경시킬 수 있고, DB의 데이터를 불러와 데이터 분석 및 시각화 등을 수월하게 진행할 수 있음

In [2]:
!pip install pymysql

In [3]:
import pymysql

### DB와 파이썬 연결 및 커서 객체 생성

In [4]:
# host : IP주소(로컬)
# user : 사용자명(HeidiSQL에서 사용자 이름과 동일)
# db : DB명칭
# charset-'utf8' : 한글 인코딩(코드를 한글 문자로 변경)
conn = pymysql.connect(host="127.0.0.1", 
                       user="root", 
                       password="1234", 
                       db="person", 
                       charset="utf8")

In [5]:
# 쿼리문의 실행 결과를 담아줄 커서 객체 생성
cur = conn.cursor()

In [6]:
# execute : 쿼리문을 실행해주는 함수(문자열로 SQL문을 입력)
temp = cur.execute('SHOW DATABASES')
temp

# 리턴값으로는 영향을 받은 행 수를 반환함

5

In [7]:
# fetchall : 쿼리 실행 결과 출력(튜플로 출력됨)
cur.fetchall()

(('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('person',),
 ('sys',))

In [8]:
print(cur.execute('SHOW TABLES'))
print(cur.fetchall())

1
(('member_list',),)


In [10]:
# 편의상 address 부터 우측 4개 컬럼 삭제
cur.execute('ALTER TABLE member_list DROP address, DROP rating, DROP first_purchase, DROP coupon')

0

### 1. SELECT문 활용

In [11]:
sql = 'SELECT * FROM member_list'
print(cur.execute(sql))
cur.fetchall()

7


((1, '최성우', '남', '28', datetime.date(2024, 1, 1)),
 (2, '김민지', '여', '26', datetime.date(2024, 1, 15)),
 (4, '이몽룡', '남', '22', datetime.date(2024, 2, 11)),
 (5, '김현지', '여', '30', datetime.date(2024, 3, 1)),
 (6, '윈터', '여', '23', datetime.date(2024, 2, 15)),
 (7, '카리나', '여', '24', datetime.date(2024, 2, 13)),
 (8, '지젤', '여', '23', datetime.date(2024, 2, 10)))

In [12]:
# 특정 테이블, 컬럼의 데이터 전체 조회 함수 선언
def search_data(table, *columns) :
    cur = conn.cursor()
    # 가변매개변수 columns는 튜플로 저장되기 때문에 복수개의 컬럼명을 문자열로 이어줌(join)
    sql = f"SELECT {','.join(columns)} FROM {table}"
    cur.execute(sql)

    return cur.fetchall()

In [15]:
search_data("member_list", "*")

((1, '최성우', '남', '28', datetime.date(2024, 1, 1)),
 (2, '김민지', '여', '26', datetime.date(2024, 1, 15)),
 (4, '이몽룡', '남', '22', datetime.date(2024, 2, 11)),
 (5, '김현지', '여', '30', datetime.date(2024, 3, 1)),
 (6, '윈터', '여', '23', datetime.date(2024, 2, 15)),
 (7, '카리나', '여', '24', datetime.date(2024, 2, 13)),
 (8, '지젤', '여', '23', datetime.date(2024, 2, 10)))

### 2. INSERT문 활용

In [16]:
no = input("번호를 입력하세요(정수 입력) : ")
name = input("이름을 입력하세요(문자열 입력) : ")
gender = input("성별을 입력하세요(남,여 중 입력) : ")
age = input("나이를 입력하세요(정수 입력) : ")
join_date = input("가입일을 입력하세요(YYYY-MM-DD 입력) : ")

sql = f"INSERT INTO member_list VALUES({no},'{name}','{gender}',{age},'{join_date}')"

cur.execute(sql)

번호를 입력하세요(정수 입력) :  9
이름을 입력하세요(문자열 입력) :  성우
성별을 입력하세요(남,여 중 입력) :  남
나이를 입력하세요(정수 입력) :  20
가입일을 입력하세요(YYYY-MM-DD 입력) :  2026-01-19


1

In [17]:
search_data("member_list", "*")

((1, '최성우', '남', '28', datetime.date(2024, 1, 1)),
 (2, '김민지', '여', '26', datetime.date(2024, 1, 15)),
 (4, '이몽룡', '남', '22', datetime.date(2024, 2, 11)),
 (5, '김현지', '여', '30', datetime.date(2024, 3, 1)),
 (6, '윈터', '여', '23', datetime.date(2024, 2, 15)),
 (7, '카리나', '여', '24', datetime.date(2024, 2, 13)),
 (8, '지젤', '여', '23', datetime.date(2024, 2, 10)),
 (9, '성우', '남', '20', datetime.date(2026, 1, 19)))

In [18]:
# 변경사항을 실제 DB에 반영(커밋)
conn.commit()

In [19]:
def insert_data(table) :
    cur = conn.cursor()
    no = input("번호를 입력하세요(정수 입력) : ")
    name = input("이름을 입력하세요(문자열 입력) : ")
    gender = input("성별을 입력하세요(남,여 중 입력) : ")
    age = input("나이를 입력하세요(정수 입력) : ")
    join_date = input("가입일을 입력하세요(YYYY-MM-DD 입력) : ")
    
    sql = f"INSERT INTO {table} VALUES({no},'{name}','{gender}',{age},'{join_date}')"
    
    result = cur.execute(sql)
    print(f"영향을 받은 행 : {result}개")
    conn.commit()

In [20]:
insert_data("member_list")

번호를 입력하세요(정수 입력) :  10
이름을 입력하세요(문자열 입력) :  나나
성별을 입력하세요(남,여 중 입력) :  여
나이를 입력하세요(정수 입력) :  20
가입일을 입력하세요(YYYY-MM-DD 입력) :  2026-01-19


영향을 받은 행 : 1개


In [21]:
search_data("member_list", "*")

((1, '최성우', '남', '28', datetime.date(2024, 1, 1)),
 (2, '김민지', '여', '26', datetime.date(2024, 1, 15)),
 (4, '이몽룡', '남', '22', datetime.date(2024, 2, 11)),
 (5, '김현지', '여', '30', datetime.date(2024, 3, 1)),
 (6, '윈터', '여', '23', datetime.date(2024, 2, 15)),
 (7, '카리나', '여', '24', datetime.date(2024, 2, 13)),
 (8, '지젤', '여', '23', datetime.date(2024, 2, 10)),
 (9, '성우', '남', '20', datetime.date(2026, 1, 19)),
 (10, '나나', '여', '20', datetime.date(2026, 1, 19)))

### 3. UPDATE문 활용
- UPDATE 사용자 정의 함수에 매개변수는 테이블명, 변경정보, 조건을 넣을 수 있도록 정의하세요.
- No9번 데이터의 성별을 반대로 바꿔보세요.

### 4. DELETE문 활용
- DELETE 사용자 정의 함수에 매개변수는 테이블명, 조건을 넣을 수 있도록 정의하세요.
- No10번 데이터를 삭제해보세요.